In [16]:
import numpy as np
import pandas as pd

In [17]:
raw_data = (pd.read_excel("C:\\SOURABH DATA\\My Computing Data.xlsx")).to_numpy()
raw_data

array([[  29.95,   73.42, 1011.44,   66.32,  432.81],
       [   5.76,   40.07, 1013.22,   62.99,  494.67],
       [  29.44,   64.33, 1011.4 ,   61.78,  442.32],
       ...,
       [  20.16,   60.1 , 1011.24,   86.23,  453.36],
       [  10.52,   41.78, 1013.54,   71.52,  474.58],
       [  16.46,   61.27, 1020.02,   74.54,  456.64]])

In [22]:
#Normalization of dataset between :
p_train= 600
p_test= 400
normalized_data = np.zeros((1000,5))
point_ones = np.ones((1000,5))/10
for x in range(5):
    in_min = np.min(raw_data[:,x:x+1])
    in_max = np.max(raw_data[:,x:x+1])
    normalized_data[:,x:x+1] = normalized_data[:,x:x+1] + point_ones[:,x:x+1] + (0.8*(raw_data[:,x:x+1] - in_min) / (in_max - in_min))  
normalized_data
pattern_input = normalized_data[:,:-1]
Target_output = normalized_data[:,-1:]

In [27]:
input_neurons = 4
hidden_neurons = 10
output_neurons = 1
eta = 0.2
a1 =1
a2 = 1

In [28]:
#V_in = (pd.read_csv("C:\\SOURABH DATA\\Soft computing data\\V.csv")).to_numpy()
#V= V_in[:,1:]
#b1_in = (pd.read_csv("C:\\SOURABH DATA\\Soft computing data\\b1.csv")).to_numpy()
#b1 = b1_in[:,1:]
#W_in = (pd.read_csv("C:\\SOURABH DATA\\Soft computing data\\W.csv")).to_numpy()
#W = W_in[:,1:]
#b2_in = (pd.read_csv("C:\\SOURABH DATA\\Soft computing data\\b2.csv")).to_numpy()
#b2 = b2_in[:,1:]
V = np.random.uniform(low=-1, high=1, size=(input_neurons,hidden_neurons))
arrVV = pd.DataFrame(V)
arrVV.to_csv("V_initial.csv")
b1 = np.random.uniform(low=-1, high=1, size=(hidden_neurons,1))
arrbb1 = pd.DataFrame(b1)
arrbb1.to_csv("b1_initial.csv")
W= np.random.uniform(low=-1, high=1, size=(hidden_neurons,output_neurons))
arrWW = pd.DataFrame(W)
arrWW.to_csv("W_initial.csv")
b2= np.random.uniform(low=-1, high=1, size=(output_neurons,1))
arrbb2 = pd.DataFrame(b2)
arrbb2.to_csv("b2_initial.csv")
delta_W = np.zeros((hidden_neurons,output_neurons))
delta_b2 = np.zeros((output_neurons,1))
delta_V = np.zeros((hidden_neurons,input_neurons))
delta_b1 = np.zeros((hidden_neurons,1))
error = 1
num = 0
mse = np.zeros((30000,1))
energy = np.zeros((30000,1))
print("initial input to hidden weights are :")
print(V)
print("initial hidden to output weights are")
print(W)
while error>0.0001:
    for p in range(p_train):
        error = 0
        error_energy = 0
        target1 = Target_output[p:p+1,:]
        T = target1.transpose()
        input1 = pattern_input[p:p+1,:]
        I = input1.transpose()   
        IH = np.matmul(V.transpose(), I) + b1
        OH = 1/(1+ np.exp(-a1*IH))
        IO = np.matmul(W.transpose(), OH) + b2
        OO = 1/(1 + np.exp(-a2*IO))
        i = 0
        j = 0
        k = 0
        while j< hidden_neurons :
            while k< output_neurons :
                delta_W = delta_W + ((T[k:,:]-OO[k:,:])*a2*OO[k:,:]*(1-OO[k:,:])*OH[:,j:])
                delta_b2 = delta_b2 + ((T[k:,:]-OO[k:,:])*a2*OO[k:,:]*(1-OO[k:,:]))
                k=k+1
            j=j+1
        delta_W = eta*delta_W
        delta_b2 = eta*delta_b2
    
        i = 0
        j = 0
        k = 0
        while i< input_neurons :
            while j< hidden_neurons :
                while k< output_neurons :
                    delta_V = delta_V + (T[k:,:]-OO[k:,:])*a2*OO[k:,:]*(1-OO[k:,:])*W[j:,k:]*a1*OH[:,j:]*(1-OH[:,j:])*((I[i:,:]).transpose())
                    delta_b1 = delta_b1 + (T[k:,:]-OO[k:,:])*a2*OO[k:,:]*(1-OO[k:,:])*W[j:,k:]*a1*OH[:,j:]*(1-OH[:,j:])
                    k=k+1
                j=j+1
            i=i+1
        delta_V = eta*delta_V
        delta_b1 = eta*delta_b1
        error = error + ((T-OO)**2)
        error_energy = error_energy + 0.5*((T-OO)**2)

    error = error/p_train
    error_energy = error_energy/p_train
    delta_V = delta_V/p_train
    IH = IH/p_train
    OH = OH/p_train
    IO = IO/p_train
    OO = OO/p_train
    delta_W= delta_W/p_train
    delta_b2 = delta_b2/p_train
    delta_V = delta_V/p_train
    delta_b1 = delta_b1/p_train
    mse[num-1:num,:] = mse[num-1:num,:] + error
    energy[num-1:num,:] = energy[num-1:num,:] + error_energy
    W = W + delta_W
    b1 = b1 + delta_b1
    V = V + delta_V.transpose()
    b2 = b2 + delta_b2
    num = num+1
    print("iteration no: {} and mean square error: {}".format(num,error))

#size_err_mse = np.size(mse_error,axis = 0)
#count = np.array([list(range(1, size_err_mse+1))]).transpose()
#my_array = np.concatenate((count,mse_error),axis = 1)
temp_mse = mse[:num,:]
arr_mse = pd.DataFrame(temp_mse) #, columns =['iteration','error'])
arr_mse.to_csv("mse.csv")
temp_energy = energy[:num,:]
arr_energy =  pd.DataFrame(temp_energy)
arr_energy.to_csv("error energy.csv")
arrV = pd.DataFrame(V)
arrV.to_csv("weights_input_to_hidden.csv")
arrb1 = pd.DataFrame(b1)
arrb1.to_csv("bias_input_to_hidden_layer.csv")
arrW = pd.DataFrame(W)
arrW.to_csv("weights_hidden_to_output.csv")
arrb2 = pd.DataFrame(b2)
arrb2.to_csv("bias_input_to_output_layer.csv")
print("mean square error: ")
print(mse)
print("error energy: ")
print(energy)
print("updated weights input to hidden")
print(V)
print("updated bias input to hidden layer")
print(b1)
print("updated weights hidden to output")
print(W)
print("updated bias input to output layer")
print(b2)
print("number of iterations are:")
print(num)


initial input to hidden weights are :
[[-0.46847041 -0.65191857 -0.77282483  0.94859225 -0.46520261 -0.81114762
   0.83991261 -0.21106087 -0.39761345 -0.09780883]
 [-0.71185158  0.88227485 -0.16706822 -0.58906613 -0.55019668  0.99852271
  -0.70448867  0.74251498 -0.80036856  0.77668049]
 [ 0.49439768 -0.4035158   0.94962554  0.4272486   0.56454542  0.77888522
   0.15357627  0.14146572  0.72337198 -0.64036997]
 [-0.93643371  0.42315139 -0.13210052 -0.46209495 -0.08206999 -0.75806247
  -0.69448233  0.86929493  0.20153483 -0.67205131]]
initial hidden to output weights are
[[-0.36065715]
 [-0.87808668]
 [-0.88833449]
 [ 0.11395653]
 [ 0.10449671]
 [ 0.66824782]
 [ 0.23542234]
 [-0.83048804]
 [-0.56819375]
 [ 0.06721122]]
iteration no: 1 and mean square error: [[2.3153303e-05]]
mean square error: 
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
error energy: 
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
updated weights input to hidden
[[-0.46847041 -0.65191856 -0.77282483  0.94859225 -0.46520261 -0.

In [ ]:
v_in = (pd.read_csv("C:\\SOURABH DATA\\Soft computing data\\weights_input_to_hidden.csv")).to_numpy()
v_test= v_in[:,1:]
print("weights input to hidden")
print(v_test)
b1_in = (pd.read_csv("C:\\SOURABH DATA\\Soft computing data\\bias_input_to_hidden_layer.csv")).to_numpy()
b1_test = b1_in[:,1:]
print("bias_input_to_hidden_layer")
print(b1_test)
w_in = (pd.read_csv("C:\\SOURABH DATA\\Soft computing data\\weights_hidden_to_output.csv")).to_numpy()
w_test = w_in[:,1:]
print("weights_hidden_to_output")
print(w_test)
b2_in = (pd.read_csv("C:\\SOURABH DATA\\Soft computing data\\bias_input_to_output_layer.csv")).to_numpy()
b2_test = b2_in[:,1:]
print("bias_input_to_output_layer")
print(b2_test)

In [ ]:
OO_test = np.zeros((p_test-1,1))
for p in range(p_test-1,1):
    target1 = Target_output[p+601:p+602,:]
    t_test = target1.transpose()
    input1 = pattern_input[p+601:p+602,:]
    I_test = input1.transpose()   
    IH_test = np.matmul(v_test.transpose(), I_test) + b1_test
    OH_test = 1/(1+ np.exp(-a1*IH_test))
    IO_test = np.matmul(w_test.transpose(), OH_test) + b2_test
    OO_test[p:p+1,:] = OO_test[p:p+1,:] + ( 1/(1 + np.exp(-a2*IO_test)))

print("output")
print(OO_test)

in_min = np.min(raw_data[:,-1:])
in_max = np.max(raw_data[:,-1:])
final_output = (((OO_test - point_ones[601:1000,-1:])/0.8)*(in_max - in_min)) + in_min
print("final output")
print(final_output)

relative_error = np.absolute(( raw_data[601:1000,-1:] - final_output[:,:] )/ raw_data[601:1000,-1:])
arr = pd.DataFrame(relative_error)
arr.to_csv("error_in_prediction.csv")
print("error of prediction")
print(relative_error)